In [16]:
import datetime as dt
import numpy as np
import pandas as pd

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

from flask import Flask, jsonify


In [17]:
#ref for check_same_thread: https://docs.python.org/3/library/sqlite3.html
engine = create_engine("sqlite:///Resources/hawaii.sqlite", connect_args={'check_same_thread': False}, echo=True)


In [18]:
# Reflect Database into ORM classes
Base = automap_base()
Base.prepare(engine, reflect=True)


2020-02-10 11:55:51,835 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-02-10 11:55:51,836 INFO sqlalchemy.engine.base.Engine ()
2020-02-10 11:55:51,837 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-02-10 11:55:51,838 INFO sqlalchemy.engine.base.Engine ()
2020-02-10 11:55:51,839 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-02-10 11:55:51,840 INFO sqlalchemy.engine.base.Engine ()
2020-02-10 11:55:51,841 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("measurement")
2020-02-10 11:55:51,842 INFO sqlalchemy.engine.base.Engine ()
2020-02-10 11:55:51,844 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'measurement' AND type = 'table'
2020-02-10 11:55:51,845 INFO sqlalchemy.engine.base.Engine ()
2020-02-10 11:55:51,847 INFO sqlalc

In [19]:
#assign measurement/stattion classes
Measurement = Base.classes.measurement
Station = Base.classes.station
#create session

session=Session(engine)

In [20]:
#Flask

app = Flask(__name__)

In [21]:
app.config["JSON_SORT_KEYS"] = False

In [22]:
@app.route("/")
def index():
    return (
        f"-------------------------<br>"
        f"Available Routes:<br>"
        f"-------------------------<br>"
        f"Precipitation for last year: /api/v1.0/precipitation<br/>"
        f"List of all stations: /api/v1.0/stations<br/>"
        f"Date and temperature observations from the last year: /api/v1.0/tobs<br/>"        
        f"Min, Avg, Max Temp given a start date up to most current date in db: /api/v1.0/2012-05-15<br/>"
        f"Min, Avg, Max Temp given a start and end date: /api/v1.0/2015-09-12/2015-09-13<br/>"
    )


In [23]:
#Convert Query to a dictionary using 'date' as 'prcp' value. Return JSON representation of dictionary.
@app.route("/api/v1.0/precipitation")    
def precip():
    results = session.query(Measurement.date, Measurement.prcp)\
    .filter(Measurement.date >= '2016-08-22')\
    .filter(Measurement.date <= '2017-08-23')\
    .order_by(Measurement.date)
    
    precip_data = []
    for r in results:
        precip_dict = {}
        precip_dict['date'] = r.date
        precip_dict['prcp'] = r.prcp
        precip_data.append(precip_dict)

    return jsonify(precip_data)


In [24]:

# Return a JSON list of stations from the dataset.
@app.route("/api/v1.0/stations")
def stations():
    #query for the data, practicing join even though station table has both columns queried below
    results = session.query(Station.name, Measurement.station)\
    .filter(Station.station == Measurement.station)\
    .group_by(Station.name).all()

    stations_data = []
    for r in results:
        stations_dict = {}
        stations_dict['name']    = r.name
        stations_dict['station'] = r.station
        stations_data.append(stations_dict)
    
    return jsonify(stations_data)


In [25]:
# Query for the dates and temperature observations from a year from the last data point. Return a JSON list of Temperature Observations (tobs) for the previous year.
@app.route("/api/v1.0/tobs")
def tobs():
    results = session.query(Measurement.date, Measurement.tobs)\
    .filter(Measurement.date >= '2016-08-22')\
    .filter(Measurement.date <= '2017-08-23')\
    .order_by(Measurement.date)

    tobs_data = []
    for r in results:
        tobs_dict = {}
        tobs_dict['date'] = r.date
        tobs_dict['tobs'] = r.tobs
        tobs_data.append(tobs_dict)
    
    return jsonify(tobs_data)


In [26]:
# When given the start only, calculate `TMIN`, `TAVG`, and `TMAX` for all dates greater than and equal to the start date.
@app.route("/api/v1.0/<start>")
def temp_stats_start(start):

    results = session.query\
    (func.min(Measurement.tobs).label('min'),\
    func.avg(Measurement.tobs).label('avg'),\
    func.max(Measurement.tobs).label('max'))\
    .filter(Measurement.date >= start).all()
    

    start_stats_data = []
    for r in results:
        start_stats_dict = {}
        start_stats_dict['Start Date'] = start
        start_stats_dict['Min Temp'] = r.min
        start_stats_dict['Avg Temp'] = r.avg
        start_stats_dict['Max Temp'] = r.max
        start_stats_data.append(start_stats_dict)
    
    return jsonify(start_stats_data)



In [27]:
# When given the start and the end date, calculate the TMIN, TAVG, and TMAX for dates between the start and end date inclusive.
@app.route("/api/v1.0/<start>/<end>")
def temp_stats_start_end(start, end):

    results = session.query(func.min(Measurement.tobs).label('min'),\
    func.avg(Measurement.tobs).label('avg'),\
    func.max(Measurement.tobs).label('max'))\
    .filter(Measurement.date >= start)\
    .filter(Measurement.date <= end).all()

    start_end_stats_data = []
    for r in results:
        start_end_stats_dict = {}
        start_end_stats_dict['Start Date'] = start
        start_end_stats_dict['End Date'] = end
        start_end_stats_dict['Min Temp'] = r.min
        start_end_stats_dict['Avg Temp'] = r.avg
        start_end_stats_dict['Max Temp'] = r.max
        start_end_stats_data.append(start_end_stats_dict)
    
    return jsonify(start_end_stats_data)

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with stat


SystemExit: 1

C:\Users\maxon\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
